In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv('../input/airline-passenger-satisfaction/train.csv')
df.head()

In [3]:
eda_df = pd.read_csv('../input/airline-passenger-satisfaction/train.csv')

In [4]:
df = df.drop(['Unnamed: 0', 'id'], axis = 1)

In [5]:
df['Gender'] = df['Gender'].replace({"Male": 0, "Female": 1})
df['Customer Type'] = df['Customer Type'].replace({"Loyal Customer": 0, "disloyal Customer": 1})
df['Type of Travel'] = df['Type of Travel'].replace({"Personal Travel": 0, "Business travel": 1})

In [6]:
print(df['Class'].unique(), df['satisfaction'].unique())

In [7]:
df['Class'] = df['Class'].replace({"Eco Plus": 0, "Business": 1, "Eco": 2})
df['satisfaction'] = df['satisfaction'].replace({"neutral or dissatisfied": 0, "satisfied": 1})

In [8]:
df.head()

In [9]:
df = df.dropna()

In [10]:
df.isna().sum()

In [11]:
df.dtypes

# Exploratory Data Analysis

In [12]:
import seaborn as sns

In [13]:
#checking frequency for different columns
sns.displot(eda_df['Gender'])
sns.displot(eda_df['Age'])
sns.displot(eda_df['Type of Travel'])
sns.displot(eda_df['Class'])
sns.displot(eda_df['Flight Distance'])

Initial Notes:
* Gender and Age distributions seem okay
* Majority of travel for business so wifi might be more important but in-flight entertainment might not be as important
* Most of the data is from business class, which has more leg room (leg room might be somewhat skewed)
* Mostly domestic/closer flights, so entertainment and leg room take a back seat

In [14]:
sns.heatmap(df.corr())

# Dataset Split & Normalization

In [15]:
X = np.array(df.drop('satisfaction', axis = 1))

In [16]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)

In [17]:
y = encoder.fit_transform(np.array(df['satisfaction']).reshape(-1, 1))

In [18]:
y

In [19]:
from sklearn.preprocessing import normalize

In [20]:
X = normalize(X)

In [21]:
X_train = X
y_train = y

# Prep Test Dataset

In [22]:
test = pd.read_csv('../input/airline-passenger-satisfaction/test.csv')

In [23]:
test = test.drop(['Unnamed: 0', 'id'], axis = 1)

In [24]:
test['Gender'] = test['Gender'].replace({"Male": 0, "Female": 1})
test['Customer Type'] = test['Customer Type'].replace({"Loyal Customer": 0, "disloyal Customer": 1})
test['Type of Travel'] = test['Type of Travel'].replace({"Personal Travel": 0, "Business travel": 1})

In [25]:
test['Class'] = test['Class'].replace({"Eco Plus": 0, "Business": 1, "Eco": 2})
test['satisfaction'] = test['satisfaction'].replace({"neutral or dissatisfied": 0, "satisfied": 1})

In [26]:
test = test.dropna()

In [27]:
X_test = test.drop('satisfaction', axis = 1)

In [28]:
y_test = encoder.fit_transform(np.array(test['satisfaction']).reshape(-1, 1))

In [29]:
X_test = normalize(X_test)

# Building Basic Model

In [30]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

In [31]:
model = Sequential()
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'sigmoid')) #output

In [32]:
model.compile(optimizer = Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [33]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

In [34]:
import matplotlib.pyplot as plt

In [35]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val accuracy'])
plt.show()

Notes on basic training:
* model is very easy to train on - 2 layer model with 5 epochs gets 80% accuracy
* no overfitting - train accuracy and val accuracy stay about the same

# Hyperparameter Tuning

**Tuning Techniques**:
Bayesian Optimization with Keras Tuner

> Important Notes: 
> Baseline: 87% Accuracy; 86% Val Accuracy
> Search Objective: Validation Accuracy

In [36]:
import keras_tuner

In [43]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(Dense(units = hp.Int(f"units_layer_{i+1}", 32, 512, 32)))
        if hp.Boolean("dropout"):
            model.add(Dropout(0.25))
    model.add(Dense(2, activation = 'sigmoid')) #output
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling = 'log')
    model.compile(optimizer = Adam(learning_rate = learning_rate), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [44]:
build_model(keras_tuner.HyperParameters())

In [39]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel = build_model,
    objective = 'val_accuracy',
    max_trials = 10,
    executions_per_trial = 1
)

In [40]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Best Model:
* num_layers: 2
* units_layer_0: 480
* dropout: False
* learning_rate: 0.00040228790416015195
* units_layer_1: 32
* Score: 0.8304947018623352

* Better accuracy than base model
* Testing with 10 epochs instead of 5

In [60]:
best_model = Sequential()
best_model.add(Dense(480, activation = 'relu'))
best_model.add(Dense(32, activation = 'relu'))
best_model.add(Dense(2, activation = 'sigmoid'))

best_model.compile(optimizer = Adam(learning_rate = 0.00040228790416015195), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [61]:
history = best_model.fit(X_train, y_train, epochs = 25, validation_data = (X_test, y_test))

In [62]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val accuracy'])
plt.show()

Final Results:
* Accuracy: 88%
* Val Accuracy: 89%